In [1]:
!pip install arxiv

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6105 sha256=266485c953ac7a944f19c06e2cdd103a2251ad61593afd33a802dea4a15cb908
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\f0\69\93\a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k

   ------------- -------------------------- 1/3 [feedparser]
   ------------- -------------------------- 1/3 [feedparser]
   ------------- -------------------------- 1/3 [feedparser]
   ------------- -------------------------- 1/3 [feedparser]
   ------------- -------------------------- 1/3 [feedparser]
   ------------- -------------------------- 1/3 [feedparser]
   -------------------------- ------------- 2/3 [arxiv]
   ---------------------------------------- 3/3 [arxiv]



  DEPRECATION: Building 'sgmllib3k' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'sgmllib3k'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [4]:
!pip install wikipedia

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11785 sha256=0e727fda60b6b20fdd5241b2919bdbeb7b90ce4550dc3e94f263603f76427c10
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\5e\b6\c5\93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


  DEPRECATION: Building 'wikipedia' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'wikipedia'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [5]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [6]:
wiki.name

'wikipedia'

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

from langchain_text_splitters import RecursiveCharacterTextSplitter


USER_AGENT environment variable not set, consider setting it to identify your requests.
c:\Users\DELL\Downloads\GitHub\LangChain-Agent\my_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents,embeddings)

retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002529321B580>, search_kwargs={})

In [10]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [11]:
retriever_tool.name

'langsmith_search'

In [12]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [13]:
tools=[wiki,arxiv,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\DELL\\Downloads\\GitHub\\LangChain-Agent\\my_env\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000025293229870>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], ve

In [14]:
from dotenv import load_dotenv

load_dotenv()
import os
os.environ['GROQ_API_KEY']=os.getenv("GROQ_API_KEY")
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-70b-8192", temperature=0)

In [17]:
from langchain import hub
# Working prompt for Groq-compatible agents (REACT format)
prompt = hub.pull("hwchase17/react")
print(prompt.template) 

c:\Users\DELL\Downloads\GitHub\LangChain-Agent\my_env\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [19]:
### Agents for open ai
# from langchain.agents import create_openai_tools_agent
# agent=create_openai_tools_agent(llm,tools,prompt)


# for groq
from langchain.agents import initialize_agent, AgentType
agent = initialize_agent(
    tools=tools,
    llm=llm,  # Your ChatGroq instance
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


C:\Users\DELL\AppData\Local\Temp\ipykernel_8652\1570934389.py:8: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [27]:
## Agent Executer
from langchain.agents import AgentExecutor
# agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
# agent_executor


In [ ]:
# response = agent.invoke({"input": "Tell me about Langsmith"})
# print(response)




> Entering new AgentExecutor chain...


IndexError: tuple index out of range

In [ ]:
print(agent.input_keys)

['', 'page_content', 'input']
